Wishes for utils:
- Load data
- Convert df to list of ents
    - Load only specific domains
- ML utils
    - Encode line using tokenizer
    - Make an actual PyTorch dataset
    - Align predictions
    - Make predictions
    - Save predictions
- Test predictions
    - Combine actual with preds (preds should already be correct format)
    - Span f1


# Load data

In [4]:
import pandas as pd
import os
import NERutils as NU

PATH = "/Users/krusand/Documents/GitHub/CrossNER-Active-Learning/data/"

train = NU.readDataset(PATH + "train.parquet")
dev = NU.readDataset(PATH + "dev.parquet")
test = NU.readDataset(PATH + "test.parquet")


# Convert to ents list

In [2]:
def entsToList(line):
    ents = line['ents']
    tokens = line['tokens']
    ents_list = []

    for token in tokens:
        start = token['start']
        end = token['end']

        if len(ents) > 0:
            ent = ents[0]
            ent_start = ent['start']
            ent_end = ent['end']
            ent_label = ent['label']

            if end == ent_end:
                ents_list.append(ent_label)
                ents = ents[1:]
            elif start >= ent_start and end < ent_end:
                ents_list.append(ent_label)
            else:
                ents_list.append('O')
                
        else:
            ents_list.append('O')
    
    return ents_list


def loadEntsForPredictions(df):
    ent_list = []
    for i in range(len(df)):
        ent_list.append(entsToList(df.iloc[i]))
    return ent_list
    

In [7]:
train_ent_list = NU.getEntsForPredictions(train)
train_ent_list

[['GPE', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'DATE',
  'O',
  'GPE',
  'GPE',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'DATE',
  'DATE',
  'DATE',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'GPE',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'NORP',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'QUANTITY',
  'QUANTITY',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'CARDINAL',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O', 'O', 'TIME', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['WORK OF ART', 'WORK OF ART', 'WORK OF ART', 'O', 'DATE', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', '

# Encoding data

In [4]:
import pandas as pd
import numpy as np
import torch


from transformers import AutoConfig
from transformers import AutoTokenizer
from transformers import Trainer
from transformers import TrainingArguments

from seqeval.metrics import f1_score


import torch.nn as nn
from transformers import BertConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.bert.modeling_bert import BertModel
from transformers.models.bert.modeling_bert import BertPreTrainedModel


In [5]:
tags, index2tag, tag2index = NU.getVocabFeatures(train)

In [7]:

max_length = NU.find_max_length(train)
filter = train['dagw_domain'] == "Web"




train_dataset = pd.DataFrame([NU.encode_line(l, max_length, tag2index) for l in train[filter].iloc])
dev_dataset = pd.DataFrame([NU.encode_line(l, max_length,tag2index) for l in dev.iloc])
test_dataset = pd.DataFrame([NU.encode_line(l, max_length,tag2index) for l in test.iloc])



Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3550, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/3h/hhvlktwj1rzfznpk5wz5w45w0000gn/T/ipykernel_70198/3127410630.py", line 7, in <module>
    train_dataset = pd.DataFrame([NU.encode_line(l, max_length, tag2index) for l in train[filter].iloc])
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/3h/hhvlktwj1rzfznpk5wz5w45w0000gn/T/ipykernel_70198/3127410630.py", line 7, in <listcomp>
    train_dataset = pd.DataFrame([NU.encode_line(l, max_length, tag2index) for l in train[filter].iloc])
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/krusand/Documents/GitHub/CrossNER-Active-Learning/utils/NERutils.py", line 55, in encode_line
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11